# Load Global Libraries

In [3]:
from signal_extraction import *
from mlutils import *
import os


In [4]:
def torque_from_ang(ang):
    m=1.12
    l=0.15
    g=9.81
    return m*g*l*np.sin(ang*np.pi/180)

# Load Data

In [5]:
subjects=os.listdir(BASE_EXTRACTION_PATH)
Fs=2000
init=False
for s in subjects[:1]:
    for f in range(2,3):
        t, ang, emg=read_extracted_combined_data(s, f)
        ang_dot=np.zeros(len(ang))
        ang_dot[:-1]=(ang[1:]-ang[:-1])*Fs
        ang_dot[-1]=ang_dot[-2]

        if not init:
            X_data=np.array(emg[:3])            
            y_data=np.array(ang).reshape((1,len(ang)))
            yv_data=np.array(ang_dot).reshape((1,len(ang_dot)))
            init=True
        else:
            X_data=np.c_[X_data,np.array(emg[:3])]
            y_data=np.c_[y_data,np.array(ang).reshape((1,len(ang)))]
            yv_data=np.c_[yv_data,np.array(ang_dot).reshape((1,len(ang_dot)))]

        y_data=torque_from_ang(y_data)

X_data=X_data.T
y_data=y_data.T
yv_data=yv_data.T
yt_data=0.4*9.8*0.25*np.sin(y_data)
print(f'{X_data.shape} and {y_data.shape}')



(69506, 3) and (69506, 1)


In [6]:
X=X_data
y=y_data

In [7]:
import matplotlib.pyplot as plt

%matplotlib qt
muscles=['Biceps Brachii', 'Triceps Lateral', 'Triceps Medial']
for muscle_index in range(3):
    plt.subplot(1,3,muscle_index+1)
    plt.scatter(X_data[:,muscle_index], y_data[:,0])
    plt.xlabel(f'{muscles[muscle_index]} Activation (% of MVC)')
    plt.ylabel('Elbow Angle')

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X,y)

X_train.shape

(52129, 3)

In [11]:
from sklearn.linear_model import LinearRegression

class PolyRegression:
    def __init__(self, powers=[1]):
        self.powers=powers
        self.model=LinearRegression()

    def fit(self, X, y):
        init=False
        for p in self.powers:
            if not init:
                X=X**p
                init=True
            else:
                X=np.c_[X,X**p]

        self.model.fit(X,y)

    def predict(self, X):
        init=False
        for p in self.powers:
            if not init:
                X=X**p
                init=True
            else:
                X=np.c_[X,X**p]

        return self.model.predict(X)


In [12]:
from collections import Counter

m=PolyRegression([4,3,2,1])
m.fit(X_train, y_train)

y_pred=m.predict(X_test)
print(np.sqrt(np.mean((y_pred-y_test)**2)))

0.25059795537986246


In [13]:
m.model.coef_

array([[-3.72935375e+06,  8.65885359e+12,  4.35783779e+08,
         3.03384092e+11,  6.53908259e+07,  7.61527657e+06,
        -1.40523907e+08, -4.18303349e+07, -3.90542869e+06,
        -6.05389695e+03,  1.91111606e+08,  2.39948511e+06,
         3.76630602e+06, -8.65885359e+12, -4.35783799e+08,
         3.03384093e+11, -6.53922749e+07, -7.62251145e+06,
        -1.40523769e+08,  4.18306529e+07,  3.90606258e+06,
        -6.05422382e+03, -1.91072787e+08,  2.39969036e+06]])

In [14]:
ranges=[np.linspace(0,0.125,100), np.linspace(0,0.6,100), np.linspace(0,0.5,100)]



for i in range(3):
    plt.subplot(1,3,i+1)
    plt.scatter(X_data[:,i], y_data[:,0])
    plt.xlabel(f'{muscles[i]} Activation (% of MVC)')
    plt.ylabel('Elbow Angle')
    x_plot=np.zeros((3,100))
    x_plot[i,:]=ranges[i]
    
    y_plot=m.predict(x_plot.T)
    plt.plot(ranges[i],y_plot)

In [16]:
import matplotlib.pyplot as plt
t,angle, emg= read_extracted_combined_data('Abhoy',2)

torque=torque_from_ang(angle)
emg=np.array(emg[:3]).T

ang_pred=m.predict(emg)
plt.plot(t,torque, t, ang_pred)
plt.legend(['Actual Angle', 'Predicted Angle'])
plt.xlabel('Time')
plt.ylabel('Angle')
plt.title('All Subjects Same Activity')

Text(0.5, 1.0, 'All Subjects Same Activity')